## Importing Libraries and Data

In [16]:
!pip install pandas
!pip install tensorflow
!pip install tensorflow_hub
!pip install tokenization
!pip install bert-tensorflow==1.0.1

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.constraints import max_norm
import tensorflow_hub as hub

import re
from bert import tokenization
import string
tf.gfile = tf.io.gfile

import os

os.getcwd()

'/content'

In [18]:
from google.colab import drive
drive.mount('/content/drive')


train = pd.read_csv("/content/drive/MyDrive/python/train.csv")
test = pd.read_csv("/content/drive/MyDrive/python/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/python/sample_submission.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Helper functions

In [19]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:

        text = tokenizer.tokenize(text)
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len

        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [20]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    dense_layer1 = Dense(units=256, activation='relu', kernel_constraint=max_norm(2.))(clf_output)
    dense_layer1 = Dropout(0.5)(dense_layer1)
    out = Dense(1, activation='sigmoid')(dense_layer1)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Data cleaning

In [21]:
def lowercase_text(text):
    return text.lower()

train.text=train.text.apply(lambda x: lowercase_text(x))
test.text=test.text.apply(lambda x: lowercase_text(x))

In [22]:
def remove_noise(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

train.text=train.text.apply(lambda x: remove_noise(x))
test.text=test.text.apply(lambda x: remove_noise(x))

In [23]:
train.text.head(5)

0    our deeds are the reason of this earthquake ma...
1                forest fire near la ronge sask canada
2    all residents asked to shelter in place are be...
3     people receive wildfires evacuation orders in...
4    just got sent this photo from ruby alaska as s...
Name: text, dtype: object

## Pre-Training (BERT)

In [24]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [25]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [26]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

## Modelling

In [27]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [28]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=8,
    callbacks=[checkpoint],
    batch_size=10
)

Epoch 1/8
609/609 [==============================] - 1297s 2s/step - loss: 0.4512 - accuracy: 0.8061 - val_loss: 0.3736 - val_accuracy: 0.8385
Epoch 2/8
609/609 [==============================] - 1255s 2s/step - loss: 0.3123 - accuracy: 0.8791 - val_loss: 0.4285 - val_accuracy: 0.8181
Epoch 3/8
609/609 [==============================] - 1257s 2s/step - loss: 0.2009 - accuracy: 0.9278 - val_loss: 0.5277 - val_accuracy: 0.8109
Epoch 4/8
609/609 [==============================] - 1256s 2s/step - loss: 0.1281 - accuracy: 0.9553 - val_loss: 0.5541 - val_accuracy: 0.8083
Epoch 5/8
609/609 [==============================] - 1257s 2s/step - loss: 0.0909 - accuracy: 0.9686 - val_loss: 0.6423 - val_accuracy: 0.8267
Epoch 6/8
609/609 [==============================] - 1256s 2s/step - loss: 0.0621 - accuracy: 0.9768 - val_loss: 0.9027 - val_accuracy: 0.8083
Epoch 7/8
609/609 [==============================] - 1258s 2s/step - loss: 0.0542 - accuracy: 0.9767 - val_loss: 1.0167 - val_accuracy: 0.8122

In [29]:
metrics=pd.DataFrame(model.history.history)
metrics

,loss,accuracy,val_loss,val_accuracy
0,0.451224,0.806076,0.373558,0.838477
1,0.312310,0.879146,0.428470,0.818122
2,0.200914,0.927750,0.527671,0.810900
3,0.128099,0.955337,0.554138,0.808273
4,0.090933,0.968637,0.642312,0.826658
5,0.062104,0.976847,0.902738,0.808273
6,0.054172,0.976683,1.016735,0.812213
7,0.058267,0.976847,0.966746,0.820092


In [30]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

In [31]:
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)